In [1]:
import pyomo.environ as pyomo
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
INPUT_PATH = "../../data/prices_energy_input.csv"

df = pd.read_csv(INPUT_PATH, index_col="time")
df

,price,input_energy
time,,
0,0.5,0.00
1,0.6,0.00
2,1.0,0.00
3,1.0,0.00
4,0.9,0.30
5,1.1,0.15
6,1.8,0.15
7,1.5,0.05
8,0.9,0.05


In [3]:
prices = {}
for t, price in enumerate(df["price"]):
    prices[t] = price
prices

{0: 0.5,
 1: 0.6,
 2: 1.0,
 3: 1.0,
 4: 0.9,
 5: 1.1,
 6: 1.8,
 7: 1.5,
 8: 0.9,
 9: 0.8,
 10: 0.7,
 11: 1.0}

In [4]:
model = pyomo.ConcreteModel()

In [5]:
# Parameters ######################
model.T = pyomo.Set(initialize=range(len(prices)))

model.soc_init = pyomo.Param(initialize=500.0)
model.soc_max = pyomo.Param(initialize=500.0)
model.soc_min = pyomo.Param(initialize=0.0)
model.sell_max = pyomo.Param(initialize=150.0)
model.input_energy_max = pyomo.Param(initialize=100)

model.price = pyomo.Param(model.T, initialize=prices)

In [6]:
# Variables #######################
model.v_sell = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)
model.v_soc = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)
model.v_input_energy = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)
model.v_is_selling = pyomo.Var(model.T, domain=pyomo.Binary)

In [7]:
# Constraints #####################
def sell_max_rule(model, t):
    return model.v_sell[t] <= model.v_is_selling[t] * model.sell_max


def soc_max_rule(model, t):
    return model.v_soc[t] <= model.soc_max


def soc_min_rule(model, t):
    return model.v_soc[t] >= model.soc_min


def energy_equlibrium_rule(model, t):
    if t == 0:
        return (
            model.soc_init + model.v_input_energy[t] - model.v_sell[t] == model.v_soc[t]
        )
    else:
        return (
            model.v_soc[t - 1] + model.v_input_energy[t] - model.v_sell[t]
            == model.v_soc[t]
        )


def input_energy_max_rule(model, t):
    return model.v_input_energy[t] <= model.input_energy_max * (
        1 - model.v_is_selling[t]
    )


model.c_sell_max = pyomo.Constraint(model.T, rule=sell_max_rule)
model.c_soc_max = pyomo.Constraint(model.T, rule=soc_max_rule)
model.c_soc_min = pyomo.Constraint(model.T, rule=soc_min_rule)
model.c_energy_equlibrium = pyomo.Constraint(model.T, rule=energy_equlibrium_rule)
model.c_input_energy_max = pyomo.Constraint(model.T, rule=input_energy_max_rule)

In [8]:
# Objective Function ##############
def objective_func(model):
    incomes = [0] * len(prices)
    for t in range(len(prices)):
        incomes[t] = (model.v_sell[t] - model.v_input_energy[t]) * model.price[t]
    return sum(incomes)


model.obj = pyomo.Objective(rule=objective_func, sense=pyomo.maximize)
solver = pyomo.SolverFactory("cbc")
result = solver.solve(model)
result


{'Problem': [{'Name': 'unknown', 'Lower bound': 760.0, 'Upper bound': 760.0, 'Number of objectives': 1, 'Number of constraints': 33, 'Number of variables': 44, 'Number of binary variables': 12, 'Number of integer variables': 12, 'Number of nonzeros': 23, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.022643566131591797}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [9]:
sell_array = np.zeros(len(prices))
for t in model.T:
    sell_array[t] = model.v_sell[t]()
np.sum(sell_array)

np.float64(750.0)

In [10]:
soc_array = np.zeros(len(prices))
for t in model.T:
    soc_array[t] = model.v_soc[t]()
soc_array

array([500., 500., 350., 350., 450., 300., 150.,   0.,   0.,  50., 150.,
         0.])

In [11]:
input_energy_array = np.zeros(len(prices))
for t in model.T:
    input_energy_array[t] = model.v_input_energy[t]()
input_energy_array

array([  0.,   0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,  50., 100.,
         0.])

In [12]:
df["soc"] = soc_array
df["sell"] = sell_array
df["input_energy"] = input_energy_array
df

,price,input_energy,soc,sell
time,,,,
0,0.5,0.0,500.0,0.0
1,0.6,0.0,500.0,0.0
2,1.0,0.0,350.0,150.0
3,1.0,0.0,350.0,0.0
4,0.9,100.0,450.0,0.0
5,1.1,0.0,300.0,150.0
6,1.8,0.0,150.0,150.0
7,1.5,0.0,0.0,150.0
8,0.9,0.0,0.0,0.0
